# 该文件未完成

## open-meteo-api 通用方法





In [1]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry


def request_openmeteo_api(latitude:list[str],
                          longitude:list[str],
                          start_date:str,
                          end_date:str,
                          daily_indicators:list[str]) -> pd.DataFrame:
    '''
    The len(latitude) and len(longitude) entered must be equal.
    Latitude would be positive (negative) when a location is in the north (south) of the equator.
    Longitude would be positive (negative) when a location is in the east (west) of the prime meridian.

    The available parameters of daily_indicators please refer: https://open-meteo.com/en/docs/historical-weather-api

}

    :param latitude:         [float, ...]
    :param longitude:        [float, ...]
    :param start_date:       "YYYY-MM-DD"
    :param end_date:         "YYYY-MM-DD"
    :param daily_indicators: [String, ...]
    :return:pd.DataFrame
    '''
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": daily_indicators
    }
    responses = openmeteo.weather_api(url, params=params)

    request_pd = pd.DataFrame()

    for response in responses:
        # 日度指标数据
        daily_datas = response.Daily()
        # 日期列
        daily_dic = {"date": pd.date_range(
            start = pd.to_datetime(daily_datas.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily_datas.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily_datas.Interval()),
            inclusive = "left"
        )}
        # 维度列、经度列
        daily_dic["latitude"] = response.Latitude()
        daily_dic["longitude"] = response.Longitude()

        # 指标列
        i = 0
        for daily_indicator in daily_indicators:
            daily_dic[daily_indicator] = daily_datas.Variables(i).ValuesAsNumpy()
            i = i + 1

        daily_pd = pd.DataFrame(data = daily_dic)
        request_pd = pd.concat([request_pd, daily_pd])

    return request_pd

In [3]:


params = (
    [0.189201, 4.90561],
    [116.900165, 100.695199],
    "2025-01-01",
    "2025-08-05",
    ["precipitation_sum", "temperature_2m_mean", "sunshine_duration"]
)

test_pd = request_openmeteo_api(*params)

test_pd

,date,latitude,longitude,precipitation_sum,temperature_2m_mean,sunshine_duration
0,2025-01-01 00:00:00+00:00,0.175747,116.887672,21.000000,26.458586,25692.144531
1,2025-01-02 00:00:00+00:00,0.175747,116.887672,2.400000,25.989836,3264.454102
2,2025-01-03 00:00:00+00:00,0.175747,116.887672,16.800001,26.364832,29998.488281
3,2025-01-04 00:00:00+00:00,0.175747,116.887672,3.600000,25.637749,18738.814453
4,2025-01-05 00:00:00+00:00,0.175747,116.887672,11.200001,26.091913,37398.625000
...,...,...,...,...,...,...
212,2025-08-01 00:00:00+00:00,4.885765,100.666664,7.300000,27.987501,37203.433594
213,2025-08-02 00:00:00+00:00,4.885765,100.666664,2.200000,28.864584,40360.160156
214,2025-08-03 00:00:00+00:00,4.885765,100.666664,2.500000,28.697916,39855.027344
215,2025-08-04 00:00:00+00:00,4.885765,100.666664,4.900000,26.856253,22457.578125
